## IMPORTER LES LIBRAIRIES

In [ ]:
from math import *
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import os
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder
from statsmodels.tsa.stattools import acf
from numpy.fft import rfft, rfftfreq
from itertools import tee
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.kernel_ridge import KernelRidge as KRR

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install --upgrade pandas
!pip install --upgrade xlrd

     |████████████████████████████████| 96 kB 2.8 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
# POUR COLAB
from google.colab import drive
drive.mount('/content/gdrive')

PATH = "/content/gdrive/MyDrive/Projet_PERCY/"

!unzip /content/gdrive/MyDrive/Projet_PERCY/data

!cp /content/gdrive/MyDrive/Projet_PERCY/uutils.py /content

from uutils import *

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  /content/gdrive/MyDrive/Projet_PERCY/data.zip
replace OK2/SCA104_OK2/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## FORMATER LES DONNEES

In [ ]:
# load data
personal_data = pd.read_excel(PATH + "epidemio_et_score.xls", header = 1, skiprows=8, index_col = 0)

# get current directory
path = os.getcwd()
print("Current Directory", path)
 
# parent directory
parent = os.path.dirname(path)
print("Parent directory", parent)

regard_dictionnaire = {"regard_contraint": "R", "regard_libre": "S"}
sensors_dictionnaire = {"tete": "00B4242D.txt", "tronc": "00B42409.txt", "pied_gauche": "00B4220C.txt", "pied_droit": "00B4220F.txt"}
signal_dictionnaire = {"acceleration": "Acc_", "free_acceleration": "FreeAcc_", "gyration": "Gyr_"}

Current Directory /content
Parent directory /


## TRAITEMENT DES VARIABLES CATEGORIELLES

In [ ]:
personal_data.loc[personal_data["Heures de vol sur les 15 derniers jours"]=="X","Heures de vol sur les 15 derniers jours"] = int(0)
personal_data.loc[personal_data["Lieu de mission sur les 6 derniers mois"]==0,"Lieu de mission sur les 6 derniers mois"] = 'X'
personal_data["Lieu de mission sur les 6 derniers mois"] = personal_data["Lieu de mission sur les 6 derniers mois"].fillna("X")
personal_data.loc[personal_data["Lequelles ? (sport, arts, multimédia, tech. de relaxation… )"]==0,"Lequelles ? (sport, arts, multimédia, tech. de relaxation… )"] = 'X'
personal_data.loc[personal_data["Type de sport ( force /endurance)"]==0,"Type de sport ( force /endurance)"] = 'X'
personal_data["Prise de médicaments sur les 15 derniers jours ? "] = personal_data["Prise de médicaments sur les 15 derniers jours ? "].fillna(0)
personal_data["Quellle classe thérapeutique ? "] = personal_data["Quellle classe thérapeutique ? "].fillna("X")
personal_data.loc[personal_data["Quellle classe thérapeutique ? "]==0,"Quellle classe thérapeutique ? "] = "X"
personal_data.loc[personal_data["Sexe"]=="F","Sexe"] = int(0)
personal_data.loc[personal_data["Sexe"]=="M","Sexe"] = int(1)
personal_data["surrentrainement"] = 0
personal_data.loc[personal_data["Score SFMS >= 14"]>=14,"surrentrainement"] = 1
personal_data.loc[personal_data["Temp moyen pour les repas sur les  15J"]=='> 1h ',"Temp moyen pour les repas sur les  15J"] = '> 1h'

categorical_features_dummy = ["Sexe",
                              "Personnel naviguant ? ",
                              "Barotraumatisme sur les 3 derniers mois ?",
                              "Prise de médicaments sur les 15 derniers jours ? ",
                              "consomation de tabac  ?"]  

categorical_ordinal = ["Grade",
                       "Date dernière mission ",
                       "Temp moyen pour les repas sur les  15J",
                       "Nombre moyen  de repas par jour sur les 15J",
                       "Temps de sommeil consécutif moyen 15J"]

for col in categorical_ordinal:
    column = personal_data[[col]]
    encoder = OrdinalEncoder()
    personal_data[col] = encoder.fit_transform(column)                      
                          
categorical_features_more = ["Heures de vol sur les 15 derniers jours",
                             "Lieu de mission sur les 6 derniers mois",
                             "Automédication ?",
                             "Réalisation d'activités pour se détendre ?",
                             "Type de sport ( force /endurance)",
                             'Quellle classe thérapeutique ? ',
                             'Lequelles ? (sport, arts, multimédia, tech. de relaxation… )']

for col in categorical_features_more:
    column = personal_data[col]
    features_names = []
    for el in np.unique(column):
        features_names.append(f"{col}_{el}")
    features_names = features_names[1:]
    new_data = pd.get_dummies(column, drop_first=True)
    new_data.columns = features_names
    personal_data = pd.concat([personal_data, new_data], axis=1)

## MODIFICATIONS A LA MAIN NECESSAIRES :

matricule 83NJ :
- le fichier SCA83j_OK2 (et tous les sous fichiers) a ete change en SCA83n_OK2 (erreur de nom de fichier)

matricule 91ND :
- le fichier ecg a ete renomme car erreur de nom : 91NC au lieu de 91ND

matricule 106TN :
- renommer la colonne dans le ecg . csv (il doit y avoir un pb d encodage)

matricule 107SM :
- renommer les fichers dans ecg . csv (1O7SM au lieu de 107SM)

### Pour generer le fichier pickle de donnees formatees, choisir une des 4 cellule suivante a lancer en focntion du pickle souhaité.

In [ ]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)
    
def get_fourier_features(signal: np.ndarray, n_bins: int = 1000) -> dict:
    """The signal is assumed to be centered and scaled to unit variance."""
    Frequency = 10
    n_samples = signal.shape[0]
    fourier = abs(rfft(signal))
    freqs = rfftfreq(n=n_samples, d=1.0 / Frequency)
    res_list = list()
    freq_bins = np.linspace(0, Frequency / 2, n_bins + 1)
    for (f_min, f_max) in pairwise(freq_bins):
        keep = (f_min <= freqs) & (freqs < f_max)
        if np.sum(fourier[keep] ** 2) > 0:
            res_list.append( np.log(np.sum(fourier[keep] ** 2)))
        else: 
            res_list.append( 0 )
    return res_list


from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y




In [ ]:
# MODIF DANS LE BUT DE REGARDER LES DONNEES DE LA MARCHE (DONNEES XCR et XCS) + ECG :

# liste des individus ou ca bloque : 

# matricule 4CS :
# pas de fichier ecg .csv : mais un fichier ecg debout + marche .csv
personal_data = personal_data[personal_data["Numéro inclusion"] != "4CS"]

# matricule 17BK :
# pas des fichier ecs . csv
personal_data = personal_data[personal_data["Numéro inclusion"] != "17BK"]

# matricule 106TN :
# pas des fichier ecs . csv
personal_data = personal_data[personal_data["Numéro inclusion"] != "106TN"]

# matricule 82GA :
# pas des fichier de fichier R
personal_data = personal_data[personal_data["Numéro inclusion"] != "82GA"]

# matricule 80DD :
# que le fichier ECG donc pas de 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "80DD"]

# matricule 87RC :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "87RC"]

# matricule 97RM :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "97RM"]

# matricule 99TB :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "99TB"]

# pas des fichier de fichier R alors que XCR==True
personal_data = personal_data[personal_data["Numéro inclusion"] != "82GA"]

# on utilise XCR/XCS :
personal_data = personal_data[(personal_data.XCR!="N") & (personal_data.XCS!="N")]

# on cree une colonne avec la cohorte
personal_data.loc[ personal_data["Score SFMS >= 14"] >= 14 , "cohorte" ] = 'SCA'
personal_data.loc[ personal_data["Score SFMS >= 14"] < 14 , "cohorte" ] = 'OK2'

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees pour chaque individu
personal_data["folder"] = personal_data.apply(lambda row: row["cohorte" ] + '/SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte"], axis=1)

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees 01_manu_steps.txt pour chaque individu
personal_data["steps_folder_S"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_00000000_XC' + "S" + '_01_' + 'manu_steps.txt', axis=1)
personal_data["steps_folder_R"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_00000000_XC' + "R" + '_01_' + 'manu_steps.txt', axis=1)

# on cree une colonne avec la liste des coordonnees x des points indiquant les pas pour chaque individu (du fichier 01_manu_steps.txt)
personal_data["steps_x_coord_S"] = personal_data.apply(lambda row: steps_manu_x(row["steps_folder_S"]), axis=1)
personal_data["steps_x_coord_R"] = personal_data.apply(lambda row: steps_manu_x(row["steps_folder_R"]), axis=1)

# on cree une colonne qui contient le dictionnaire qui donne pour chaque capteur le nom du fichier dans lequel trouver les donnees pour chaque individu
personal_data["dict_capteur_S"] = personal_data.apply(lambda row: \
    {"nom_tete" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tete'], \
    "nom_tronc" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tronc'], \
    "nom_pied_gauche" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_gauche'], \
    "nom_pied_droit" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_droit'] }, axis=1)
personal_data["dict_capteur_R"] = personal_data.apply(lambda row: \
    {"nom_tete" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['tete'], \
    "nom_tronc" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['tronc'], \
    "nom_pied_gauche" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['pied_gauche'], \
    "nom_pied_droit" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['pied_droit'] }, axis=1)

# Setting standard filter requirements.
order = 6
fs = 30.0
cutoff = 3.667

# on cree une colonne avec les moyenne / variance des signaux de l acceleration, free_acceleration, gyration
dict_capteur_column_S = np.where(personal_data.columns=="dict_capteur_S")[0].item()
for key in personal_data.iloc[0,dict_capteur_column_S].keys():
    for ref in ["Acc_X", "Acc_Y", "Acc_Z", "FreeAcc_X", "FreeAcc_Y", "FreeAcc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]:
        personal_data[key+ref+"clean"] = personal_data.apply(lambda row: butter_lowpass_filter(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref], cutoff, fs, order), axis=1)
        personal_data[ key + "_mean_S_" + ref ] = personal_data.apply(lambda row: np.mean(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_std_S_" + ref ] = personal_data.apply(lambda row: np.std(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_min_S_" + ref ] = personal_data.apply(lambda row: np.min(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_max_S_" + ref ] = personal_data.apply(lambda row: np.max(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_freq_S_" + ref ] = personal_data.apply(lambda row: get_fourier_features( np.array(list(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref])) ), axis=1)

dict_capteur_column_R = np.where(personal_data.columns=="dict_capteur_R")[0].item()
for key in personal_data.iloc[0,dict_capteur_column_R].keys():
    for ref in ["Acc_X", "Acc_Y", "Acc_Z", "FreeAcc_X", "FreeAcc_Y", "FreeAcc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]:
        personal_data[key+ref+"clean"] = personal_data.apply(lambda row: butter_lowpass_filter(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref], cutoff, fs, order), axis=1)
        personal_data[ key + "_mean_R_" + ref ] = personal_data.apply(lambda row: np.mean(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_std_R_" + ref ] = personal_data.apply(lambda row: np.std(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_min_R_" + ref ] = personal_data.apply(lambda row: np.min(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_max_R_" + ref ] = personal_data.apply(lambda row: np.max(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_freq_R_" + ref ] = personal_data.apply(lambda row: get_fourier_features( np.array(list(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref])) ), axis=1)

personal_data["ecg_folder"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_ecg/' + (row["Numéro inclusion"]) + '.csv', axis=1)
personal_data["ecg_data"] = personal_data.apply(lambda row: ECG_RR(row["ecg_folder"]), axis=1)

personal_data["ecg_mean"] = personal_data.apply(lambda row: np.nanmean(row["ecg_data"]), axis=1)
personal_data["ecg_std"] = personal_data.apply(lambda row: np.nanstd(row["ecg_data"]), axis=1)
personal_data["ecg_min"] = personal_data.apply(lambda row: np.nanmin(row["ecg_data"]), axis=1)
personal_data["ecg_max"] = personal_data.apply(lambda row: np.nanmax(row["ecg_data"]), axis=1)
personal_data["ecg_acf"] = personal_data.apply(lambda row: acf(row["ecg_data"], missing="conservative"), axis=1)

steps_S  = personal_data.steps_x_coord_S.to_numpy()
max_step_S = np.max(np.max(steps_S))
min_step_S = np.min(np.min(np.min(steps_S)))
steps_right_S = [ (steps_S[i][0] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_left_S = [ (steps_S[i][1] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_right_S_mean = np.array([np.mean(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_right_S_std = np.array([np.std(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_left_S_mean = np.array([np.mean(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_left_S_std = np.array([np.std(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_diff_S = abs(steps_right_S_mean - steps_left_S_mean)

personal_data["steps_right_S_mean"] = steps_right_S_mean
personal_data["steps_right_S_std"] = steps_right_S_std
personal_data["steps_left_S_mean"] = steps_left_S_mean
personal_data["steps_left_S_std"] = steps_left_S_std
personal_data["steps_diff_S"] = steps_diff_S

steps_R  = personal_data.steps_x_coord_R.to_numpy()
max_step_R = np.max(np.max(steps_R))
min_step_R = np.min(np.min(np.min(steps_R)))
steps_right_R = [ (steps_R[i][0] - min_step_R)/(max_step_R - min_step_R) for i in range(len(steps_R))]
steps_left_R = [ (steps_R[i][1] - min_step_R)/(max_step_R - min_step_R) for i in range(len(steps_R))]
steps_right_R_mean = np.array([np.mean(steps_right_R[i])  for i in range(len(steps_right_R))])
steps_right_R_std = np.array([np.std(steps_right_R[i])  for i in range(len(steps_right_R))])
steps_left_R_mean = np.array([np.mean(steps_left_R[i])  for i in range(len(steps_left_R))])
steps_left_R_std = np.array([np.std(steps_left_R[i])  for i in range(len(steps_left_R))])
steps_diff_R = abs(steps_right_R_mean - steps_left_R_mean)

personal_data["steps_right_R_mean"] = steps_right_R_mean
personal_data["steps_right_R_std"] = steps_right_R_std
personal_data["steps_left_R_mean"] = steps_left_R_mean
personal_data["steps_left_R_std"] = steps_left_R_std
personal_data["steps_diff_R"] = steps_diff_R

personal_data["steps_right_diffSR_mean"] = abs(steps_right_R_mean - steps_right_S_mean)
personal_data["steps_right_diffSR_std"] = abs(steps_right_R_std - steps_right_S_std)
personal_data["steps_left_diffSR_mean"] = abs(steps_left_R_mean - steps_left_S_mean)
personal_data["steps_left_diffSR_std"] = abs(steps_left_R_std - steps_left_S_std)
personal_data["steps_diff_SR"] = abs(steps_diff_R - steps_diff_S)

# personal_data.to_pickle(PATH + "complete_personal_data_marche_XCR_XCS_ECG.pkl")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DtypeWarning: Columns (0,1,2,3,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DtypeWarning: Columns (0,1,2,3,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DtypeWarning: Columns (0,1,2,3,6,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DtypeWarning: Columns (0,1,2,3,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/ipyke

In [ ]:
personal_data.to_pickle(PATH + "complete_personal_data_marche_XCR_XCS_ECG.pkl")

In [ ]:
# MODIF DANS LE BUT DE REGARDER LES DONNEES DE LA MARCHE (DONNEES XCR et XCS):

# liste des individus ou ca bloque : 

# matricule 80DD :
# que le fichier ECG donc pas de 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "80DD"]

# matricule 87RC :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "87RC"]

# matricule 97RM :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "97RM"]

# matricule 99TB :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "99TB"]

# pas des fichier de fichier R alors que XCR==True
personal_data = personal_data[personal_data["Numéro inclusion"] != "82GA"]

# on utilise XCR/XCS :
personal_data = personal_data[(personal_data.XCR!="N") & (personal_data.XCS!="N")]

# on cree une colonne avec la cohorte
personal_data.loc[ personal_data["Score SFMS >= 14"] >= 14 , "cohorte" ] = 'SCA'
personal_data.loc[ personal_data["Score SFMS >= 14"] < 14 , "cohorte" ] = 'OK2'

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees pour chaque individu
personal_data["folder"] = personal_data.apply(lambda row: row["cohorte" ] + '/SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte"], axis=1)

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees 01_manu_steps.txt pour chaque individu
personal_data["steps_folder_S"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_00000000_XC' + "S" + '_01_' + 'manu_steps.txt', axis=1)
personal_data["steps_folder_R"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_00000000_XC' + "R" + '_01_' + 'manu_steps.txt', axis=1)

# on cree une colonne avec la liste des coordonnees x des points indiquant les pas pour chaque individu (du fichier 01_manu_steps.txt)
personal_data["steps_x_coord_S"] = personal_data.apply(lambda row: steps_manu_x(row["steps_folder_S"]), axis=1)
personal_data["steps_x_coord_R"] = personal_data.apply(lambda row: steps_manu_x(row["steps_folder_R"]), axis=1)

# on cree une colonne qui contient le dictionnaire qui donne pour chaque capteur le nom du fichier dans lequel trouver les donnees pour chaque individu
personal_data["dict_capteur_S"] = personal_data.apply(lambda row: \
    {"nom_tete" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tete'], \
    "nom_tronc" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tronc'], \
    "nom_pied_gauche" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_gauche'], \
    "nom_pied_droit" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_droit'] }, axis=1)
personal_data["dict_capteur_R"] = personal_data.apply(lambda row: \
    {"nom_tete" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['tete'], \
    "nom_tronc" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['tronc'], \
    "nom_pied_gauche" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['pied_gauche'], \
    "nom_pied_droit" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "R" + '_01_' + sensors_dictionnaire['pied_droit'] }, axis=1)

# on cree une colonne avec les moyenne / variance des signaux de l acceleration, free_acceleration, gyration
dict_capteur_column_S = np.where(personal_data.columns=="dict_capteur_S")[0].item()
for key in personal_data.iloc[0,dict_capteur_column_S].keys():
    for ref in ["Acc_X", "Acc_Y", "Acc_Z", "FreeAcc_X", "FreeAcc_Y", "FreeAcc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]:
        personal_data[ key + "_mean_S_" + ref ] = personal_data.apply(lambda row: np.mean(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_std_S_" + ref ] = personal_data.apply(lambda row: np.std(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_min_S_" + ref ] = personal_data.apply(lambda row: np.min(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_max_S_" + ref ] = personal_data.apply(lambda row: np.max(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)

dict_capteur_column_R = np.where(personal_data.columns=="dict_capteur_R")[0].item()
for key in personal_data.iloc[0,dict_capteur_column_R].keys():
    for ref in ["Acc_X", "Acc_Y", "Acc_Z", "FreeAcc_X", "FreeAcc_Y", "FreeAcc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]:
        personal_data[ key + "_mean_R_" + ref ] = personal_data.apply(lambda row: np.mean(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_std_R_" + ref ] = personal_data.apply(lambda row: np.std(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_min_R_" + ref ] = personal_data.apply(lambda row: np.min(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)
        personal_data[ key + "_max_R_" + ref ] = personal_data.apply(lambda row: np.max(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_R"][key])[ref]), axis=1)

steps_S  = personal_data.steps_x_coord_S.to_numpy()
max_step_S = np.max(np.max(steps_S))
min_step_S = np.min(np.min(np.min(steps_S)))
steps_right_S = [ (steps_S[i][0] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_left_S = [ (steps_S[i][1] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_right_S_mean = np.array([np.mean(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_right_S_std = np.array([np.std(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_left_S_mean = np.array([np.mean(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_left_S_std = np.array([np.std(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_diff_S = abs(steps_right_S_mean - steps_left_S_mean)

personal_data["steps_right_S_mean"] = steps_right_S_mean
personal_data["steps_right_S_std"] = steps_right_S_std
personal_data["steps_left_S_mean"] = steps_left_S_mean
personal_data["steps_left_S_std"] = steps_left_S_std
personal_data["steps_diff_S"] = steps_diff_S

steps_R  = personal_data.steps_x_coord_R.to_numpy()
max_step_R = np.max(np.max(steps_R))
min_step_R = np.min(np.min(np.min(steps_R)))
steps_right_R = [ (steps_R[i][0] - min_step_R)/(max_step_R - min_step_R) for i in range(len(steps_R))]
steps_left_R = [ (steps_R[i][1] - min_step_R)/(max_step_R - min_step_R) for i in range(len(steps_R))]
steps_right_R_mean = np.array([np.mean(steps_right_R[i])  for i in range(len(steps_right_R))])
steps_right_R_std = np.array([np.std(steps_right_R[i])  for i in range(len(steps_right_R))])
steps_left_R_mean = np.array([np.mean(steps_left_R[i])  for i in range(len(steps_left_R))])
steps_left_R_std = np.array([np.std(steps_left_R[i])  for i in range(len(steps_left_R))])
steps_diff_R = abs(steps_right_R_mean - steps_left_R_mean)

personal_data["steps_right_R_mean"] = steps_right_R_mean
personal_data["steps_right_R_std"] = steps_right_R_std
personal_data["steps_left_R_mean"] = steps_left_R_mean
personal_data["steps_left_R_std"] = steps_left_R_std
personal_data["steps_diff_R"] = steps_diff_R

personal_data["steps_right_diffSR_mean"] = abs(steps_right_R_mean - steps_right_S_mean)
personal_data["steps_right_diffSR_std"] = abs(steps_right_R_std - steps_right_S_std)
personal_data["steps_left_diffSR_mean"] = abs(steps_left_R_mean - steps_left_S_mean)
personal_data["steps_left_diffSR_std"] = abs(steps_left_R_std - steps_left_S_std)
personal_data["steps_diff_SR"] = abs(steps_diff_R - steps_diff_S)

personal_data.to_pickle(PATH + "complete_personal_data_marche_XCR_XCS.pkl")

In [ ]:
# MODIF DANS LE BUT DE REGARDER LES DONNEES DE LA MARCHE (XCS):

# liste des individus ou ca bloque : 

# matricule 80DD :
# que le fichier ECG donc pas de 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "80DD"]

# matricule 87RC :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "87RC"]

# matricule 97RM :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "97RM"]

# matricule 99TB :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "99TB"]

# pas des fichier de fichier R alors que XCR==True
personal_data = personal_data[personal_data["Numéro inclusion"] != "82GA"]

# on utilise XCS :
personal_data = personal_data[personal_data.XCS=="O"]

# on cree une colonne avec la cohorte
personal_data.loc[ personal_data["Score SFMS >= 14"] >= 14 , "cohorte" ] = 'SCA'
personal_data.loc[ personal_data["Score SFMS >= 14"] < 14 , "cohorte" ] = 'OK2'

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees pour chaque individu
personal_data["folder"] = personal_data.apply(lambda row: row["cohorte" ] + '/SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte"], axis=1)

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees 01_manu_steps.txt pour chaque individu
personal_data["steps_folder_S"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_00000000_XC' + "S" + '_01_' + 'manu_steps.txt', axis=1)

# on cree une colonne avec la liste des coordonnees x des points indiquant les pas pour chaque individu (du fichier 01_manu_steps.txt)
personal_data["steps_x_coord_S"] = personal_data.apply(lambda row: steps_manu_x(row["steps_folder_S"]), axis=1)

# on cree une colonne qui contient le dictionnaire qui donne pour chaque capteur le nom du fichier dans lequel trouver les donnees pour chaque individu
personal_data["dict_capteur_S"] = personal_data.apply(lambda row: \
    {"nom_tete" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tete'], \
    "nom_tronc" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tronc'], \
    "nom_pied_gauche" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_gauche'], \
    "nom_pied_droit" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_droit'] }, axis=1)

# on cree une colonne avec les moyenne / variance des signaux de l acceleration, free_acceleration, gyration
dict_capteur_column_S = np.where(personal_data.columns=="dict_capteur_S")[0].item()
for key in personal_data.iloc[0,dict_capteur_column_S].keys():
    for ref in ["Acc_X", "Acc_Y", "Acc_Z", "FreeAcc_X", "FreeAcc_Y", "FreeAcc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]:
        personal_data[ key + "_mean_S_" + ref ] = personal_data.apply(lambda row: np.mean(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_std_S_" + ref ] = personal_data.apply(lambda row: np.std(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_min_S_" + ref ] = personal_data.apply(lambda row: np.min(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_max_S_" + ref ] = personal_data.apply(lambda row: np.max(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)

steps_S  = personal_data.steps_x_coord_S.to_numpy()
max_step_S = np.max(np.max(steps_S))
min_step_S = np.min(np.min(np.min(steps_S)))
steps_right_S = [ (steps_S[i][0] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_left_S = [ (steps_S[i][1] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_right_S_mean = np.array([np.mean(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_right_S_std = np.array([np.std(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_left_S_mean = np.array([np.mean(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_left_S_std = np.array([np.std(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_diff_S = abs(steps_right_S_mean - steps_left_S_mean)

personal_data["steps_right_S_mean"] = steps_right_S_mean
personal_data["steps_right_S_std"] = steps_right_S_std
personal_data["steps_left_S_mean"] = steps_left_S_mean
personal_data["steps_left_S_std"] = steps_left_S_std
personal_data["steps_diff_S"] = steps_diff_S

personal_data.to_pickle(PATH + "complete_personal_data_marche_XCS.pkl")

In [ ]:
# MODIF DANS LE BUT DE REGARDER LES DONNEES DES ECG + LA MARCHE (XCS):

# liste des individus ou ca bloque : 

# matricule 80DD :
# que le fichier ECG
personal_data = personal_data[personal_data["Numéro inclusion"] != "80DD"]

# matricule 87RC :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "87RC"]

# matricule 97RM :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "97RM"]

# matricule 99TB :
# pas de fichier avec les points du type 01_manu_steps.txt
personal_data = personal_data[personal_data["Numéro inclusion"] != "99TB"]

# matricule 4CS :
# pas de fichier ecg .csv : mais un fichier ecg debout + marche .csv
personal_data = personal_data[personal_data["Numéro inclusion"] != "4CS"]

# matricule 17BK :
# pas des fichier ecs . csv
personal_data = personal_data[personal_data["Numéro inclusion"] != "17BK"]

# matricule 106TN :
# pas des fichier ecs . csv
personal_data = personal_data[personal_data["Numéro inclusion"] != "106TN"]

# matricule 82GA :
# pas des fichier de fichier R
personal_data = personal_data[personal_data["Numéro inclusion"] != "82GA"]

# on utilise XCS :
personal_data = personal_data[personal_data.XCS=="O"]

# on cree une colonne avec la cohorte
personal_data.loc[ personal_data["Score SFMS >= 14"] >= 14 , "cohorte" ] = 'SCA'
personal_data.loc[ personal_data["Score SFMS >= 14"] < 14 , "cohorte" ] = 'OK2'

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees pour chaque individu
personal_data["folder"] = personal_data.apply(lambda row: row["cohorte" ] + '/SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte"], axis=1)

# on cree une colonne avec le nom du folder dans lequel se trouvent les donnees 01_manu_steps.txt pour chaque individu
personal_data["steps_folder_S"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_00000000_XC' + "S" + '_01_' + 'manu_steps.txt', axis=1)

# on cree une colonne avec la liste des coordonnees x des points indiquant les pas pour chaque individu (du fichier 01_manu_steps.txt)
personal_data["steps_x_coord_S"] = personal_data.apply(lambda row: steps_manu_x(row["steps_folder_S"]), axis=1)

# on cree une colonne qui contient le dictionnaire qui donne pour chaque capteur le nom du fichier dans lequel trouver les donnees pour chaque individu
personal_data["dict_capteur_S"] = personal_data.apply(lambda row: \
    {"nom_tete" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tete'], \
    "nom_tronc" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['tronc'], \
    "nom_pied_gauche" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_gauche'], \
    "nom_pied_droit" : 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + \
     '_00000000_XC' + "S" + '_01_' + sensors_dictionnaire['pied_droit'] }, axis=1)

# on cree une colonne avec les moyenne / variance des signaux de l acceleration, free_acceleration, gyration
dict_capteur_column_S = np.where(personal_data.columns=="dict_capteur_S")[0].item()
for key in personal_data.iloc[0,dict_capteur_column_S].keys():
    for ref in ["Acc_X", "Acc_Y", "Acc_Z", "FreeAcc_X", "FreeAcc_Y", "FreeAcc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z"]:
        personal_data[ key + "_mean_S_" + ref ] = personal_data.apply(lambda row: np.mean(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_std_S_" + ref ] = personal_data.apply(lambda row: np.std(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_min_S_" + ref ] = personal_data.apply(lambda row: np.min(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)
        personal_data[ key + "_max_S_" + ref ] = personal_data.apply(lambda row: np.max(importfile_XSens(row["folder" ] + '/' + row["dict_capteur_S"][key])[ref]), axis=1)

personal_data["ecg_folder"] = personal_data.apply(lambda row: row["folder" ] + '/' + 'SCA' + (row["Numéro inclusion"][0:3]).lower() + '_' + row["cohorte" ] + '_ecg/' + (row["Numéro inclusion"]) + '.csv', axis=1)
personal_data["ecg_data"] = personal_data.apply(lambda row: ECG_RR(row["ecg_folder"]), axis=1)

personal_data["ecg_mean"] = personal_data.apply(lambda row: np.nanmean(row["ecg_data"]), axis=1)
personal_data["ecg_std"] = personal_data.apply(lambda row: np.nanstd(row["ecg_data"]), axis=1)
personal_data["ecg_min"] = personal_data.apply(lambda row: np.nanmin(row["ecg_data"]), axis=1)
personal_data["ecg_max"] = personal_data.apply(lambda row: np.nanmax(row["ecg_data"]), axis=1)
personal_data["ecg_acf"] = personal_data.apply(lambda row: acf(row["ecg_data"], missing="conservative"), axis=1)

steps_S  = personal_data.steps_x_coord_S.to_numpy()
max_step_S = np.max(np.max(steps_S))
min_step_S = np.min(np.min(np.min(steps_S)))
steps_right_S = [ (steps_S[i][0] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_left_S = [ (steps_S[i][1] - min_step_S)/(max_step_S - min_step_S) for i in range(len(steps_S))]
steps_right_S_mean = np.array([np.mean(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_right_S_std = np.array([np.std(steps_right_S[i])  for i in range(len(steps_right_S))])
steps_left_S_mean = np.array([np.mean(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_left_S_std = np.array([np.std(steps_left_S[i])  for i in range(len(steps_left_S))])
steps_diff_S = abs(steps_right_S_mean - steps_left_S_mean)

personal_data["steps_right_S_mean"] = steps_right_S_mean
personal_data["steps_right_S_std"] = steps_right_S_std
personal_data["steps_left_S_mean"] = steps_left_S_mean
personal_data["steps_left_S_std"] = steps_left_S_std
personal_data["steps_diff_S"] = steps_diff_S

personal_data.to_pickle(PATH + "complete_personal_data_marche_XCS_ECG.pkl")